In [3]:
import chromadb
from chromadb.utils import embedding_functions
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
import joblib

In [39]:
df = pd.read_csv("../chunked_data.csv" , nrows=1000)

In [40]:
df.shape

(1000, 5)

In [41]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [42]:
device

device(type='cuda')

In [43]:
# Initialize the SentenceTransformer model
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
model.to(device)  # Move model to GPU if available

# Define a function to generate BERT embeddings
def get_bert_embeddings(text):
    with torch.no_grad():
        embeddings = model.encode([text])
    return embeddings

# Add BERT embeddings column to the DataFrame
df['bert_embedding'] = df['chunked_text'].apply(get_bert_embeddings)

#for 5 roes = 2.5 seconds
#for 10 rows = 2.5 seconds
#for 20 rows = 


In [45]:
df.tail()

,num,name,content,file_content,chunked_text,bert_embedding
995,9185838,the.rising.s01.e05.episode.1.5.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x002\xba\x99V...,maria all i can think about is neve joan whate...,['maria all i can think about is neve joan wha...,"[[-0.5765835, -0.14024329, 0.28995615, -0.4889..."
996,9185839,the.rising.s01.e06.episode.1.6.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00#\xba\x99V...,alex he cant even see you neve he can now i do...,['alex he cant even see you neve he can now i ...,"[[-0.39805153, -0.09775657, 0.27896366, -0.035..."
997,9185840,the.rising.s01.e07.episode.1.7.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xea\xa9\x...,we found traces of blood in room seven of keat...,['we found traces of blood in room seven of ke...,"[[-0.2904275, -0.4023079, 0.4591143, -0.758200..."
998,9185841,the.rising.s01.e08.episode.1.8.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xba\x...,alex neve neve stop it neve victoria you were ...,['alex neve neve stop it neve victoria you wer...,"[[-0.43899527, -0.44110787, 0.058933605, 0.009..."
999,9185844,the.rising.s01.e01.episode.1.1.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x0e\xaa\x...,use the free code joinnow at wwwplayshipseu he...,['use the free code joinnow at wwwplayshipseu ...,"[[0.09849142, -0.08207184, 0.5981183, -0.61691..."


In [ ]:
import chromadb
from chromadb.utils import embedding_functions
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.externals import joblib
import torch

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the dataset into a DataFrame (adjust nrows as needed)
nrows = 5000
df = pd.read_csv('your_dataset.csv', nrows=nrows)

# Initialize the SentenceTransformer model
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
model.to(device)  # Move model to GPU if available

# Define a function to generate BERT embeddings
def get_bert_embeddings(text):
    with torch.no_grad():
        embeddings = model.encode([text])
    return embeddings

# Add BERT embeddings column to the DataFrame
df['bert_embedding'] = df['chunked_text'].apply(get_bert_embeddings)

# Save the DataFrame with embeddings to a file
joblib.dump(df, 'embeddings.pkl')
print("Embeddings saved successfully.")

# Initialize ChromaDB client
chroma_client = chromadb.PersistentClient(path="my_chromadb")

# Define embedding function
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="distilbert-base-nli-mean-tokens")

# Create or get collection
collection = chroma_client.get_or_create_collection(name="my_collection", embedding_function=sentence_transformer_ef)

# Add documents and embeddings to the collection
documents = df['name'].tolist()
embeddings = df['bert_embedding'].tolist()
metadatas = df[['name', 'content']].to_dict(orient='records')
ids = df.index.tolist()

collection.add(
    documents=documents,
    embeddings=embeddings,
    metadatas=metadatas,
    ids=ids
)

# Query for documents similar to a given text
query_text = "your query goes here"
results = collection.query(
    query_texts=[query_text],
    n_results=10,
    include=['documents', 'distances', 'metadatas']
)

print(results['documents'])
